In [1]:
!pip install midiSynth
from midiSynth.synth import MidiSynth
midi_synth = MidiSynth()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.7 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=0c9161cafbec8c66f1dc691e6255e11dd95cee44b9a063988eb6cc766c82588c
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty-midi


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -r /content/drive/MyDrive/EMOPIA-main/requirements.txt

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/EMOPIA-main/workspace/transformer')  # Change to your directory path

In [7]:
import os
import pickle
import torch

In [8]:
!pip install --user pytorch-fast-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.6 MB/s eta 0:00:00
  Created wheel for pytorch-fast-transformers: filename=pytorch_fast_transformers

In [9]:
import sys
sys.path.append("/root/.local/lib/python3.11/site-packages")  # Ensure Python can find the package

import fast_transformers
print(fast_transformers.__file__)  # This should print the module's location


/root/.local/lib/python3.11/site-packages/fast_transformers/__init__.py


In [20]:
!pwd

/content


In [14]:
from utils import write_midi
from models import TransformerModel, network_paras

## 1. Prepare dictionary

In [15]:
!gdown --id 17dKUf33ZsDbHC5Z6rkQclge3ppDTVCMP
!unzip co-representation.zip -d ../../dataset/
!rm co-representation.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=17dKUf33ZsDbHC5Z6rkQclge3ppDTVCMP
From (redirected): https://drive.google.com/uc?id=17dKUf33ZsDbHC5Z6rkQclge3ppDTVCMP&confirm=t&uuid=45c5226a-5899-4af7-93c8-693619ad61ed
To: /content/co-representation.zip
100% 16.6M/16.6M [00:00<00:00, 80.7MB/s]
Archive:  co-representation.zip
  inflating: ../../dataset/co-representation/ailabs_data.npz  
  inflating: ../../dataset/co-representation/ailabs_fn2idx_map.json  
  inflating: ../../dataset/co-representation/dictionary.pkl  
  inflating: ../../dataset/co-representation/emopia_data.npz  
  inflating: ../../dataset/co-representation/emopia_fn2idx_map.json  
  inflating: ../../dataset/co-representation/emopia_idx.npz  
  inflating: ../../dataset/co-representation/REA

In [16]:
path_dictionary = '../../dataset/co-representation/dictionary.pkl'
assert os.path.exists(path_dictionary)

In [17]:
dictionary = pickle.load(open(path_dictionary, 'rb'))
event2word, word2event = dictionary

In [18]:
# config
n_class = []   # num of classes for each token
for key in event2word.keys():
    n_class.append(len(dictionary[0][key]))
n_token = len(n_class)

## 2. Prepare model

In [19]:
os.getcwd()

'/content'

In [38]:
!gdown --id 19Seq18b2JNzOamEQMG1uarKjj27HJkHu --output exp/pretrained_transformer.zip
!unzip exp/pretrained_transformer.zip -d exp/
!rm exp/pretrained_transformer.zip
os.listdir('/content')

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Error:

	[Errno 2] No such file or directory: 'exp'

To report issues, please visit https://github.com/wkentaro/gdown/issues.
unzip:  cannot find or open exp/pretrained_transformer.zip, exp/pretrained_transformer.zip.zip or exp/pretrained_transformer.zip.ZIP.
rm: cannot remove 'exp/pretrained_transformer.zip': No such file or directory


['.config', 'model_weights.pth', 'drive', 'test.mid', 'sample_data']

In [39]:
path_saved_ckpt = '/content/drive/MyDrive/EMOPIA-main/exp/pretrained_transformer/loss_25_params.pt'
assert os.path.exists(path_saved_ckpt)

In [43]:
net.load_state_dict(torch.load(path_saved_ckpt, map_location=device), strict=False)


<All keys matched successfully>

In [48]:
# Initialize model
net = TransformerModel(n_class, is_training=False)

# Move model to CPU or GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

# Load the model weights with strict=False
#path_saved_ckpt = "path/to/your/model_checkpoint.pth"  # Ensure this path is correct
path_saved_ckpt = '/content/drive/MyDrive/EMOPIA-main/exp/pretrained_transformer/loss_25_params.pt'
net.load_state_dict(torch.load(path_saved_ckpt, map_location=device), strict=False)

# Set model to evaluation mode
net.eval()


>>>>>: [56, 135, 18, 4, 87, 18, 42, 5]
 [o] using RNN backend.


TransformerModel(
  (loss_func): CrossEntropyLoss()
  (word_emb_tempo): Embeddings(
    (lut): Embedding(56, 128)
  )
  (word_emb_chord): Embeddings(
    (lut): Embedding(135, 256)
  )
  (word_emb_barbeat): Embeddings(
    (lut): Embedding(18, 64)
  )
  (word_emb_type): Embeddings(
    (lut): Embedding(4, 32)
  )
  (word_emb_pitch): Embeddings(
    (lut): Embedding(87, 512)
  )
  (word_emb_duration): Embeddings(
    (lut): Embedding(18, 128)
  )
  (word_emb_velocity): Embeddings(
    (lut): Embedding(42, 128)
  )
  (word_emb_emotion): Embeddings(
    (lut): Embedding(5, 128)
  )
  (pos_emb): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (in_linear): Linear(in_features=1376, out_features=512, bias=True)
  (transformer_encoder): RecurrentTransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x RecurrentTransformerEncoderLayer(
        (attention): RecurrentAttentionLayer(
          (inner_attention): RecurrentLinearAttention(
            (feature_map): Ac

In [47]:
"""# init model
net = TransformerModel(n_class, is_training=False)
net.cuda()
net.eval()

net.load_state_dict(torch.load(path_saved_ckpt))"""
"""import torch

# Initialize model
net = TransformerModel(n_class, is_training=False)

# Move model to CPU instead of CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

net.eval()
"""

>>>>>: [56, 135, 18, 4, 87, 18, 42, 5]
 [o] using RNN backend.


TransformerModel(
  (loss_func): CrossEntropyLoss()
  (word_emb_tempo): Embeddings(
    (lut): Embedding(56, 128)
  )
  (word_emb_chord): Embeddings(
    (lut): Embedding(135, 256)
  )
  (word_emb_barbeat): Embeddings(
    (lut): Embedding(18, 64)
  )
  (word_emb_type): Embeddings(
    (lut): Embedding(4, 32)
  )
  (word_emb_pitch): Embeddings(
    (lut): Embedding(87, 512)
  )
  (word_emb_duration): Embeddings(
    (lut): Embedding(18, 128)
  )
  (word_emb_velocity): Embeddings(
    (lut): Embedding(42, 128)
  )
  (word_emb_emotion): Embeddings(
    (lut): Embedding(5, 128)
  )
  (pos_emb): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (in_linear): Linear(in_features=1376, out_features=512, bias=True)
  (transformer_encoder): RecurrentTransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x RecurrentTransformerEncoderLayer(
        (attention): RecurrentAttentionLayer(
          (inner_attention): RecurrentLinearAttention(
            (feature_map): Ac

## 3. Start generating

### 3.1 setup parameters

In [52]:
emotion_tag = 1  # the target emotion class you want. It should belongs to [1,2,3,4].
path_outfile = 'class2' # output midi file name

In [53]:
import torch

# Define the path where you want to save the weights
model_path = "/content/model_weights.pth"

# Save the model state dict
torch.save(net.state_dict(), model_path)

# Download the file
from google.colab import files
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
#device = torch.device("cuda")
res, _ = net.inference_from_scratch(dictionary, emotion_tag, n_token=8, display=False)
write_midi(res, path_outfile + '.mid', word2event)

#midi_synth.play_midi(path_outfile + '.mid')
#midi_synth.midi2audio(path_outfile + '.mid', path_outfile + '.mp3')

------ initiate ------
0               | 0               | 0               | Emotion         | 0               | 0               | 0               | 1               | 
0               | 0               | Bar             | Metrical        | 0               | 0               | 0               | 0               | 
------ generate ------

--------[Done]--------
(390, 8)
